In [1]:
import os 
from glob import glob
import pdal 
import laspy
import numpy as np 

def pdal_quick_info(laz_path):
    reader = pdal.Reader(laz_path) # sensitive to resolution, can pass argument here 
    pipeline = reader.pipeline()
    qi = pipeline.quickinfo 
    return qi,pipeline

/home/ljp238/miniconda3/envs/samlidar/lib/python3.9/site-packages/pyproj/__init__.py:95: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


In [2]:
eba_laz_dir = "/media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/POINTCLOUD/comprexn"
eba_tif_dir = "/media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF"
os.makedirs(eba_tif_dir, exist_ok=True)

locations = os.listdir(eba_laz_dir)
location = locations[-1]
location

'transectos_tocantins'

In [3]:
loc_laz_files = glob(f"{eba_laz_dir}/{location}/*.laz")
loc_tif_dir = f"{eba_tif_dir}/{location}"
os.makedirs(loc_tif_dir, exist_ok=True)

In [19]:
for laz in loc_laz_files:
    btif = os.path.basename(laz).replace('.laz', '.tif')
    tif = f"{loc_tif_dir}/{btif}"

laz,tif

('/media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/POINTCLOUD/comprexn/transectos_tocantins/NP_T-0171.laz',
 '/media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF/transectos_tocantins/NP_T-0171.tif')

In [20]:
# does the data contain class 2, fi so use it

In [21]:
q, p = pdal_quick_info(laz)

In [22]:
def pdal_pc_classes(pipeline):
    pipeline.execute()
    arrays = pipeline.arrays
    if len(arrays) == 0:
        print("No point data found in the file.")
    
    classification_values = arrays[0]['Classification']
    unique_classes = np.unique(classification_values)
    print(f"Unique classifications: {unique_classes}")
    return unique_classes

def laspy_pc_classes(laz_path):
    las = laspy.read(laz_path)
    unique_classes = np.unique(las.classification)
    print(f"Unique classifications: {unique_classes}")
    return unique_classes

In [23]:
#pdal_pc_classes(p) # 26s
#laspy_pc_classes(laz) # 43s


In [24]:
q, p = pdal_quick_info(laz)

In [25]:
import time 

In [ ]:
def generate_dtm(laz, tif, res=10,outfn="mean"):
    ti = time.perf_counter()

    tif = tif.replace('.tif', f"_DTM{str(res)}_{outfn}.tif")

    if os.path.isfile(tif):
        print(f"file alredy exist...\n{tif}")
        return tif 
    """Extracts ground points and generates a DTM."""
    print(f"Generating DTM: {laz} \n-> {tif}")

    pipeline = pdal.Reader(laz)
    pipeline |= pdal.Filter.expression(expression="Classification == 2")
    pipeline |= pdal.Writer.gdal(
        filename=tif, gdalopts="tiled=yes, compress=deflate", nodata=-9999,
        output_type=outfn, resolution=res
    )
    
    pipeline.execute()
    tf = time.perf_counter() - ti 
    print(f"DTM saved to {tif}")
    print(f"run.time = {tf/60} min(s)")
    return tif

In [31]:
laz = '/media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/POINTCLOUD/comprexn/transectos_tocantins/NP_T-0171.laz'
tif = '/media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF/transectos_tocantins/NP_T-0171.tif'
tif = generate_dtm(laz, tif,res=10,outfn="min")

Generating DTM: /media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/POINTCLOUD/comprexn/transectos_tocantins/NP_T-0171.laz 
-> /media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF/transectos_tocantins/NP_T-0171_DTM10_min.tif
DTM saved to /media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF/transectos_tocantins/NP_T-0171_DTM10_min.tif
run.time = 0.33909538596635685 min(s)


In [32]:
laz = '/media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/POINTCLOUD/comprexn/transectos_tocantins/NP_T-0171.laz'
tif = '/media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF/transectos_tocantins/NP_T-0171.tif'
tif = generate_dtm(laz, tif,res=10,outfn="idw")

Generating DTM: /media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/POINTCLOUD/comprexn/transectos_tocantins/NP_T-0171.laz 
-> /media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF/transectos_tocantins/NP_T-0171_DTM10_idw.tif
DTM saved to /media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF/transectos_tocantins/NP_T-0171_DTM10_idw.tif
run.time = 0.4485181365496828 min(s)


In [33]:
laz = '/media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/POINTCLOUD/comprexn/transectos_tocantins/NP_T-0171.laz'
tif = '/media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF/transectos_tocantins/NP_T-0171.tif'
tif = generate_dtm(laz, tif,res=10,outfn="mean")

Generating DTM: /media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/POINTCLOUD/comprexn/transectos_tocantins/NP_T-0171.laz 
-> /media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF/transectos_tocantins/NP_T-0171_DTM10_mean.tif
DTM saved to /media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF/transectos_tocantins/NP_T-0171_DTM10_mean.tif
run.time = 0.3414133377664257 min(s)


In [34]:
laz = '/media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/POINTCLOUD/comprexn/transectos_tocantins/NP_T-0171.laz'
tif = '/media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF/transectos_tocantins/NP_T-0171.tif'
tif = generate_dtm(laz, tif,res=10,outfn="max")

Generating DTM: /media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/POINTCLOUD/comprexn/transectos_tocantins/NP_T-0171.laz 
-> /media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF/transectos_tocantins/NP_T-0171_DTM10_max.tif
DTM saved to /media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF/transectos_tocantins/NP_T-0171_DTM10_max.tif
run.time = 0.34538042806671 min(s)


In [36]:
# ask Rolf's opition on this - I am going with iwd here. could use max as chm if anything 
# min is just a line , no much variablility at all
# mean is the winner here 

In [39]:
import os
import glob

# Define the folder path
folder_path = "/media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/POINTCLOUD/comprexy"

# Get all zip files in the folder
zip_files = glob.glob(os.path.join(folder_path, "*.zip"))
zip_files = glob.glob(os.path.join(folder_path, "*.zip"))

# Sort files by size (smallest to largest)
sorted_files = sorted(zip_files, key=os.path.getsize)
bnames = []
# Print sorted file names with sizes
for file in sorted_files:
    bname = os.path.basename(file)[:-4]
    bnames.append(bname)
    print(f"{bname}: {os.path.getsize(file) / (1024 * 1024):.2f} MB")


transectos_tocantins: 218.72 MB
transectos_para_p5: 659.20 MB
transectos_para_p7: 1193.47 MB
transectos_para_p6: 1376.32 MB
transectos_para_p8: 1575.13 MB
transectos_para_p4: 1851.38 MB
transectos_acre: 4771.86 MB
transectos_maranhao: 4957.41 MB
transectos_amapa: 6376.53 MB
transectos_roraima: 10531.11 MB
transectos_para_p3: 10801.47 MB
transectos_para_p2: 12120.47 MB
transectos rondonia: 15001.54 MB
transectos_mato_grosso: 26008.55 MB
transectos_para_p1: 30090.37 MB
transectos_amazonas_p2: 33063.28 MB
transectos_amazonas_p1: 34049.77 MB


In [40]:
bnames

['transectos_tocantins',
 'transectos_para_p5',
 'transectos_para_p7',
 'transectos_para_p6',
 'transectos_para_p8',
 'transectos_para_p4',
 'transectos_acre',
 'transectos_maranhao',
 'transectos_amapa',
 'transectos_roraima',
 'transectos_para_p3',
 'transectos_para_p2',
 'transectos rondonia',
 'transectos_mato_grosso',
 'transectos_para_p1',
 'transectos_amazonas_p2',
 'transectos_amazonas_p1']